In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
%run ../script/config.py

In [36]:
my_models_paths = !ls -d -1 ../data/models/*.txt

my_models_paths

['../data/models/4 - Baseline - Just join triples_e089e6d2328637218601e68ee1216f624e70d78f.txt',
 '../data/models/5 - Model - Template Based - roots_68ac39512ab8a4825bd4174fc6e2e1165446677e.txt']

In [37]:
models_paths = my_models_paths + SUBMISSIONS_FILEPATHS
models_paths

['../data/models/4 - Baseline - Just join triples_e089e6d2328637218601e68ee1216f624e70d78f.txt',
 '../data/models/5 - Model - Template Based - roots_68ac39512ab8a4825bd4174fc6e2e1165446677e.txt',
 '../data/webnlg2017/submissions/adaptCenter/ADAPTcentreWebNLGsubmission.txt',
 '../data/webnlg2017/submissions/melbourne/final_result.txt',
 '../data/webnlg2017/submissions/pkuwriter/PKUWriter_results.txt',
 '../data/webnlg2017/submissions/tilburg/nmt_test.out.ordered',
 '../data/webnlg2017/submissions/tilburg/smt_test.out.ordered',
 '../data/webnlg2017/submissions/tilburg/template_test.out.ordered',
 '../data/webnlg2017/submissions/uit-danglt-clnlp/Submission-UIT-DANGNT-CLNLP.txt',
 '../data/webnlg2017/submissions/upf/UPF_All_sent_final.txt',
 '../data/webnlg2017/submissions/baseline_sorted.txt']

In [38]:
model_results = {}

for model_path in models_paths:
    
    with open(model_path) as f:
        
        model_results[model_path] = [line[:-1] for line in f]

In [6]:
%run ../script/webnlg.py

In [7]:
import pandas as pd
pd.set_option('max_colwidth', 1000)
from nltk.translate.bleu_score import sentence_bleu

lex = WebNLGCorpus.load("test_with_lex")

s_multiwordexpressions = lex.mdf.m_subject.str.lower().str.replace('_', ' ').str.split().tolist()
o_multiwordexpressions = lex.mdf.m_object.str.lower().str.replace('_', ' ').str.split().tolist()

all_multiwordexpressions = s_multiwordexpressions + o_multiwordexpressions
all_multiwordexpressions = list(set([tuple(s) for s in all_multiwordexpressions]))

from nltk.tokenize.mwe import MWETokenizer

t = MWETokenizer(all_multiwordexpressions)

from textacy.preprocess import preprocess_text

def preprocess(ref):
    
    ref = preprocess_text(ref, no_accents=True, lowercase=True, no_punct=True)
    
    return t.tokenize(ref.split())


def print_entry_and_models(e):
    
    print(e)
    print('\n')
    
    idx = int(e.idx().split('_')[1])
    
    references = [preprocess(ref) for ref in e.ldf.ltext.tolist()]
    
    texts = []
    bleus = []
    
    for model_path in models_paths:
        
        model_text = model_results[model_path][idx]
        
        bleu = sentence_bleu(references, preprocess(model_text))
        
        texts.append(model_text)
        bleus.append(bleu)
        
    return pd.DataFrame({'models': models_paths,
                         'texts': texts,
                         'bleu': bleus}).sort_values('bleu', ascending=False)

In [8]:
test_with_lex = WebNLGCorpus.load('test_with_lex')

In [39]:
e = test_with_lex.sample()

df = print_entry_and_models(e)

Triple info: {'category': 'Artist', 'eid': 'Id1461', 'idx': '0_1460', 'ntriples': 3}

	Modified triples:

Albennie_Jones | genre | Jazz
Jazz | stylisticOrigin | Blues
Jazz | musicFusionGenre | Afrobeat


	Lexicalizations:

Albennie Jones plays Jazz music which originated from Blues music. A fusion of Jazz is Afrobeat music.
Albennie Jones is a Jazz musician. Jazz comes from Blues music and the Afrobeat is a fusion genre of that music.
Albennie Jones plays jazz which originates from Blues music. Afrobeat music also has elements of Jazz.




In [40]:
df

,bleu,models,texts
4,3.359973e-01,../data/webnlg2017/submissions/pkuwriter/PKUWriter_results.txt,"albennie jones belongs to the genre jazz , which originated from blues and was a fusion genre of afrobeat ."
6,1.832557e-01,../data/webnlg2017/submissions/tilburg/smt_test.out.ordered,albennie jones is a jazz jazz stylisticorigin blues jazz musicfusiongenre afrobeat
1,8.596275e-155,../data/models/5 - Model - Template Based - roots_68ac39512ab8a4825bd4174fc6e2e1165446677e.txt,"Albennie Jones is in the genre of Jazz. Jazz stylistic Origin Blues , is in the genre of Afrobeat."
5,6.590068e-155,../data/webnlg2017/submissions/tilburg/nmt_test.out.ordered,albennie jones is located in the city of afrobeat . albennie jones is affiliated with jazz in blues .
9,5.574804e-155,../data/webnlg2017/submissions/upf/UPF_All_sent_final.txt,"albennie jones plays for jazz , blues has which influenced . it has influenced afrobeat ."
3,4.276455e-155,../data/webnlg2017/submissions/melbourne/final_result.txt,albennie jones ( jazz ) is served by afrobeat and is blues .
2,2.104296e-155,../data/webnlg2017/submissions/adaptCenter/ADAPTcentreWebNLGsubmission.txt,the commeat is the genre of the genre of the genre of the genre of the genre of the genre of the genre of the genre of the genre of the genre of the genre of the genre of the genre of the genre of the genre of the genre of the genre of the genre of the genre of the genre of the genre of the genre of the genre of the genre of the genre of the genre of the genre of the genre of the genre of the genre of the genre of the genre of the g
7,2.297401e-156,../data/webnlg2017/submissions/tilburg/template_test.out.ordered,albennie jones is jazz .
0,8.451929e-232,../data/models/4 - Baseline - Just join triples_e089e6d2328637218601e68ee1216f624e70d78f.txt,Albennie Jones genre Jazz Jazz stylisticOrigin Blues Jazz musicFusionGenre Afrobeat
10,7.107197e-232,../data/webnlg2017/submissions/baseline_sorted.txt,the port authority of the republic of ireland is the aircrafthelicopter .


In [29]:
%run ../script/data_alignment.py

import spacy

nlp = spacy.load('en_core_web_lg')

In [42]:
%run ../script/data_alignment.py

In [43]:
da = RootDataAlignmentModel(similarity_metric=similarity.token_sort_ratio)

In [49]:
first_lexicalization = e.ldf.ltext.values.tolist()[1]
# uses the first triple
first_triple = e.mdf[['m_subject', 'm_object']].to_dict(orient='records')[0]

da.render_aligned(nlp(first_lexicalization), first_triple)